In [1]:
# # !gunzip cx-corpora.en2es.text.json.gz
# !pip install --user pybind11
# !pip install fasttext
#!conda install -c conda-forge fasttext

In [73]:
import json
import fasttext
import pandas as pd
import numpy as np
import re
import os
import unicodedata
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from numpy.core.umath_tests import inner1d

In [29]:
!pip install tqdm

In [33]:
def apply_transform(vec, transform):
    """
    Apply the given transformation to the vector space

    Right-multiplies given transform with embeddings E:
        E = E * transform

    Transform can either be a string with a filename to a
    text file containing a ndarray (compat. with np.loadtxt)
    or a numpy ndarray.
    """
    transmat = np.loadtxt(transform)# if isinstance(transform, str) else transform
    return np.matmul(vec, transmat)

def make_training_matrices(source_dictionary, target_dictionary, bilingual_dictionary):
    """
    Source and target dictionaries are the FastVector objects of
    source/target languages. bilingual_dictionary is a list of 
    translation pair tuples [(source_word, target_word), ...].
    """
    source_matrix = []
    target_matrix = []
    
    len_bd = len(bilingual_dictionary)

    for i, (source, target) in tqdm(enumerate(bilingual_dictionary)):
#         tqdm(f'\r{i + 1}/{len_bd} | {100 * (i + 1) / len_bd:.3f} %', end = '', flush = True)
        sourceVector = source_dictionary.get_sentence_vector(source.lower().strip().replace('_',' '))
        targetVector = target_dictionary.get_sentence_vector(target.lower().strip().replace('_',' '))
        source_matrix.append(sourceVector)
        target_matrix.append(targetVector)
        
    # return training matrices
    return np.array(source_matrix), np.array(target_matrix)

def normalized(a, axis=-1, order=2):
    """Utility function to normalize the rows of a numpy array."""
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)


def learn_transformation(source_matrix, target_matrix, normalize_vectors=True):
    """
    Source and target matrices are numpy arrays, shape
    (dictionary_length, embedding_dimension). These contain paired
    word vectors from the bilingual dictionary.
    """
    # optionally normalize the training vectors
    if normalize_vectors:
        source_matrix = normalized(source_matrix)
        target_matrix = normalized(target_matrix)

    # perform the SVD
    product = np.matmul(target_matrix.transpose(), source_matrix)
    U, s, V = np.linalg.svd(product)

    # return orthogonal transformation which aligns source language to the target
    return np.matmul(U, V)

In [4]:
# file = ('cx-corpora.en2es.text.json')

# with open(file, encoding='utf-8') as f:
#     d = json.load(f)

In [5]:
model = {}
model['en'] = fasttext.load_model('wiki.en.bin')
model['es'] = fasttext.load_model('wiki.es.bin')

In [6]:
data = pd.read_csv('descriptions_final.csv', error_bad_lines=False)


In [12]:
data = data.drop(['Unnamed: 0'], axis = 1)

In [13]:
data = data.drop_duplicates()

In [15]:
data.to_csv('descriptions_final.csv')

In [9]:
data.head()

,Unnamed: 0,source_language,source_value,target_language,target_value
0,0,en,how the player was acquired; qualifier for P54...,es,forma en que fue adquirido el jugador; calific...
1,1,en,rivers and other outflows waterway names. If e...,es,río que drena el lago
2,2,en,amount of goods and services bought from other...,es,cantidad de bienes y servicios comprados a otr...
3,3,en,Wikimedia list related to this subject,es,lista de Wikimedia para el elemento
4,4,en,identifier for a unique bibliographic record i...,es,número de control del Online Computer Library ...


In [14]:
len(data)

666870

In [16]:
data_cleaned = []
for source, target in zip(data['source_value'], data['target_value']):
    try:
        source_cleaned = unicodedata.normalize('NFD', source)
        target_cleaned = unicodedata.normalize('NFD', target)
        if not source == ''  and not target == '':
            data_cleaned.append([source_cleaned, target_cleaned])
    except:
        pass

In [17]:
len(data_cleaned)

666867

In [18]:
## Code to split train and test
df = pd.DataFrame(data_cleaned,columns = ['source','target'])
# train, test = train_test_split(df, test_size=0.2, random_state=42)

In [19]:
df.to_csv('description_train.csv')

In [20]:
# test.to_csv('content_trans_test.csv',index=False)
# train.to_csv('content_trans_train.csv',index=False)
test = pd.read_csv('../ContentTranslation/content_test.csv') #same for everyone
train = pd.read_csv('description_train.csv') #replace with your respective dataset

In [34]:
#Train
bilingual_dictionary = list(zip(train['source'],train['target']))
source_matrix, target_matrix = make_training_matrices(model['en'], model['es'], bilingual_dictionary)

666867it [01:24, 7873.06it/s]


In [37]:
transform = learn_transformation(source_matrix, target_matrix)

In [38]:
# np.savetxt('description_trans_transform_en_es.txt', transform)

In [74]:
print("Before trans:", np.mean(inner1d(target_matrix, source_matrix)))

Before trans: 0.01143723951168304


In [75]:
print("After trans:", np.mean(inner1d(target_matrix, np.matmul(transform, source_matrix.T).T)))

After trans: 0.32593447005301746


In [76]:
#Test
bilingual_dictionary = list(zip(test['source'],test['target']))
source_matrix_test, target_matrix_test = make_training_matrices(model['en'], model['es'], bilingual_dictionary)

72972it [01:28, 821.72it/s] 


In [77]:
#before
print("Before trans:",np.mean(inner1d(target_matrix_test, source_matrix_test)))
#after
print("After trans:", np.mean(inner1d(target_matrix_test, np.matmul(transform, source_matrix_test.T).T)))

Before trans: 0.0010129306058742473
After trans: 0.264224146733514
